In [1]:
import os

from dotenv import load_dotenv

from clients import OpenAILLMClient, TemplateStore

load_dotenv()

True

In [2]:
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [3]:
llm_client = OpenAILLMClient(
    key=AZURE_OPENAI_API_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
    model=AZURE_OPENAI_DEPLOYMENT_NAME,
)

In [4]:
from clients.data_clients import TemplateStore

template_store = TemplateStore(dir_path="templates", do_validation=True)

In [5]:
from flows.retrieval_eval_flow import retrieval_eval_flow

dict_eval_ret = retrieval_eval_flow(
    question="What is the capital of France?",
    answer_gt="The capital of France is Paris.",
    answer_pred="France's capital city is Paris, known for the Eiffel Tower.",
    context=[
        "Paris is the capital and most populous city of France.",
        "The Eiffel Tower, located in Paris, is a famous landmark.",
        "France is a country in Western Europe known for its art, culture, and cuisine.",
        "Lyon and Marseille are also major French cities but not the capital.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)

print(dict_eval_ret)

{'relevancy': 1.0}


In [6]:
import os

from promptflow.tracing import start_trace

from flows.generation_eval_flow import generation_eval_flow

os.environ["PF_DISABLE_TRACING"] = "FALSE"
start_trace(collection="trace_rag_metrics")

dict_eval_gen = generation_eval_flow(
    question="What is the capital of France?",
    answer_gt="The capital of France is Paris.",
    answer_pred="France's capital city is Paris, known for the Eiffel Tower.",
    context=[
        "Paris is the capital and most populous city of France.",
        "The Eiffel Tower, located in Paris, is a famous landmark.",
        "France is a country in Western Europe known for its art, culture, and cuisine.",
        "Lyon and Marseille are also major French cities but not the capital.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)

print(dict_eval_gen)

Prompt flow service has started...
{'faithfulness': 1.0, 'precision': 1.0, 'recall': 1.0}


You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0xb180b2ed63dbef0388fdef26600d518e


In [7]:
from flows.retrieval_eval_flow import retrieval_eval_flow

dict_eval_ret = retrieval_eval_flow(
    question="What are the main ingredients in traditional Italian pesto?",
    answer_gt="Traditional Italian pesto is made with basil, garlic, pine nuts, Parmesan cheese, and olive oil.",
    answer_pred="Italian pesto is usually made with spinach, almonds, cheddar cheese, and olive oil.",
    context=[
        "Pesto alla Genovese, the traditional Italian pesto, is made with fresh basil leaves, garlic, pine nuts, Parmesan cheese, and extra virgin olive oil.",
        "Spinach and other ingredients may be used in variations of pesto, but they are not part of the traditional recipe.",
        "The use of almonds or cheddar cheese is not typical in authentic Italian pesto recipes.",
        "Sunflower oil is not used in traditional Italian pesto.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)
print(dict_eval_ret)

{'relevancy': 0.25}


In [8]:
import os

from promptflow.tracing import start_trace

os.environ["PF_DISABLE_TRACING"] = "FALSE"


start_trace()
dict_eval_gen = generation_eval_flow(
    question="What are the main ingredients in traditional Italian pesto?",
    answer_gt="Traditional Italian pesto is made with basil, garlic, pine nuts, Parmesan cheese, and olive oil.",
    answer_pred="Italian pesto is usually made with spinach, almonds, cheddar cheese, and olive oil.",
    context=[
        "Pesto alla Genovese, the traditional Italian pesto, is made with fresh basil leaves, garlic, pine nuts, Parmesan cheese, and extra virgin olive oil.",
        "Spinach and other ingredients may be used in variations of pesto, but they are not part of the traditional recipe.",
        "The use of almonds or cheddar cheese is not typical in authentic Italian pesto recipes.",
        "Sunflower oil is not used in traditional Italian pesto.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)
print(dict_eval_gen)

Prompt flow service has started...
{'faithfulness': 0.25, 'precision': 0.25, 'recall': 0.0}
